In [1]:
from newspaper import Article, build
import re
from datetime import datetime
import locale
import pandas as pd
import os
from urllib.parse import urlsplit, urlunsplit
import csv
import time


In [2]:
# Lista de URLs de los sitios de noticias
urls = [
    "https://www.elcomercio.es/",
    "https://www.lavozdeasturias.es/",
    "https://www.elpais.com/",
    "https://www.abc.es/",
    "https://www.lne.es/",
    "https://www.eldiario.es/",
    "https://www.elmundo.es/",
    "https://www.larazon.es/"
    
]


In [3]:

# Lista de patrones que quieres eliminar
patrones_no_deseados = [
    r'Comenta', 
    r'Necesitas ser suscriptor', 
    r'Inicia sesión', 
    r'Opciones para compartir', 
    r'(WhatsApp|Facebook|X \(antes Twitter\)|LinkedIn|Telegram|Threads)', 
    # Expresión regular para capturar la frase "Este contenido es exclusivo para suscriptores"
    r"Este contenido es exclusivo para suscriptores\.?\s+¿Ya eres suscriptor\/a\? Inicia sesión.*",
    # Expresión regular para capturar la frase "Este vídeo es exclusivo para suscriptores"
    r"Este vídeo es exclusivo para suscriptores\. Disfruta de acceso ilimitado",
    # Expresión regular para el nuevo mensaje que quieres eliminar
    r"No se puede iniciar sesión sin correo electrónico! Por favor, compruebe su cuenta social y dar el permiso para utilizar la información de su correo electrónico.*"
]


In [4]:

# Función para limpiar el bloque no deseado y eliminar líneas vacías
def limpiar_bloque_suscriptor(texto):
    # Primero eliminamos todo lo que aparece después de "Temas"
    texto = re.sub(r"(?s)Temas.*", '', texto, flags=re.DOTALL)

    # Dividimos el texto en líneas
    lineas = texto.splitlines()

    # Filtramos las líneas que no coincidan con ninguno de los patrones y que no estén vacías o solo contengan espacios
    #texto_limpio = '\n'.join([linea.strip() for linea in lineas if linea.strip() and not any(re.search(patron, linea) for patron in patrones_no_deseados)])
    # Filtramos las líneas que no coincidan con ninguno de los patrones y que no estén vacías o solo contengan espacios
    texto_limpio = '\n'.join([linea.strip() for linea in lineas if linea.strip() and not any(re.search(patron, linea) for patron in patrones_no_deseados)])
    
    return texto_limpio


#### MEJOR OPCION, MENOR TIEMPO, Y FECHA DESPUES DE FECHAS MA RECIENTE DE CSV EXISTENTES

In [5]:

# Definir la carpeta donde están los archivos CSV
carpeta_csv = 'archivos'

# Obtener la fecha de hoy
hoy = datetime.now().strftime('%Y-%m-%d')

# Inicializar el contador total de artículos
total_articulos = 0

# Crear un conjunto para almacenar las URLs de los archivos CSV anteriores
urls_pasadas = set()
fecha_mas_reciente = None  # Variable para almacenar la fecha más reciente de los artículos previos
url_mas_reciente = None  # URL del artículo con la fecha más reciente
archivo_mas_reciente = None  # Nombre del archivo CSV donde se encontró la fecha más reciente

# Función para normalizar URLs eliminando el fragmento y los parámetros innecesarios
def normalizar_url(url):
    partes_url = urlsplit(url)
    url_sin_fragmento = partes_url._replace(fragment="")  # Eliminar fragmento
    return urlunsplit(url_sin_fragmento)

# Función para cargar las URLs y fechas de los archivos CSV anteriores
def cargar_urls_y_fecha_mas_reciente():
    global fecha_mas_reciente, url_mas_reciente, archivo_mas_reciente
    archivos_csv = [f for f in os.listdir(carpeta_csv) if f.endswith('.csv') and f != f'articulos_{hoy}.csv']
    for archivo_csv in archivos_csv:
        ruta_completa = os.path.join(carpeta_csv, archivo_csv)
        with open(ruta_completa, mode='r', newline='', encoding='utf-8') as archivo_csv_pasado:
            lector_csv = csv.reader(archivo_csv_pasado)
            next(lector_csv)  # Saltar la cabecera del archivo CSV
            for fila in lector_csv:

                url_normalizada = normalizar_url(fila[2])  # La URL está en la tercera columna (índice 2)
                urls_pasadas.add(url_normalizada)

                # Convertir la fecha (segunda columna) a datetime y verificar si es la más reciente
                try:
                    fecha = datetime.strptime(fila[1], '%Y-%m-%d')
                    if fecha_mas_reciente is None or fecha > fecha_mas_reciente:
                        fecha_mas_reciente = fecha
                        url_mas_reciente = url_normalizada
                        archivo_mas_reciente = archivo_csv
                except ValueError:
                    pass  # Ignorar fechas inválidas o desconocidas
        print(f"Cargadas URLs y fechas desde el archivo: {archivo_csv}")

# Cargar las URLs y la fecha más reciente de los archivos CSV anteriores
cargar_urls_y_fecha_mas_reciente()
print(f"Total de URLs cargadas desde archivos anteriores: {len(urls_pasadas)}")
print(f"Fecha más reciente en los archivos anteriores: {fecha_mas_reciente}")
print(f"URL de la fecha más reciente: {url_mas_reciente}")
print(f"Archivo CSV de la fecha más reciente: {archivo_mas_reciente}")

# Lista temporal para almacenar los artículos antes de ordenarlos
articulos = []

# Capturar el tiempo de inicio de la ejecución
start_time = time.time()

# Recorrer todas las URLs de la lista
for url in urls:
    print(f"\nProcesando sitio: {url}")
    try:
        paper = build(url, language='es', memoize_articles=False)
        for article in paper.articles:
            url_articulo = normalizar_url(article.url)  # Normalizar la URL del artículo
            
            # Comprobar si la URL ya está en los archivos CSV anteriores
            if url_articulo in urls_pasadas:
                print(f"Artículo ya procesado previamente, ignorado: {url_articulo}")
                continue  # Si ya está en urls_pasadas, saltar este artículo

            try:
                # Descargar y procesar el artículo solo si no ha sido procesado antes
                article.download()
                article.parse()

                titulo = article.title
                fecha_publicacion = article.publish_date

                # Si la fecha de publicación es None o inválida, asignar 'Desconocido'
                if not fecha_publicacion:
                    fecha_publicacion = 'Desconocido'
                else:
                    # Convertir a string para asegurarnos de que se pueda comparar
                    fecha_publicacion = fecha_publicacion.strftime('%Y-%m-%d')

                    # Si hay una fecha más reciente registrada y este artículo es más antiguo, ignorarlo
                    # Pero si el artículo tiene la misma fecha que la más reciente, procesarlo.
                    if fecha_mas_reciente and datetime.strptime(fecha_publicacion, '%Y-%m-%d') < fecha_mas_reciente:
                        print(f"Artículo ignorado porque es más antiguo que la fecha más reciente registrada.")
                        continue

                # Añadir el artículo a la lista si pasa todas las validaciones
                articulos.append((titulo, fecha_publicacion, url_articulo))

            except Exception as e:
                print(f"Error procesando el artículo: {e}")

    except Exception as e:
        print(f"Error procesando el sitio: {e}")

# Ordenar los artículos, manejando correctamente 'Desconocido' para que se vaya al final
articulos.sort(key=lambda x: datetime.strptime(x[1], '%Y-%m-%d') if x[1] != 'Desconocido' else datetime.max)

# Crear el archivo CSV de hoy y abrirlo en modo escritura
archivo_hoy = os.path.join(carpeta_csv, f'articulos_{hoy}.csv')
with open(archivo_hoy, mode='w', newline='', encoding='utf-8') as archivo_csv_hoy:
    escritor_csv = csv.writer(archivo_csv_hoy)
    escritor_csv.writerow(['Título', 'Fecha de Publicación', 'URL'])

    for titulo, fecha_publicacion, url_articulo in articulos:
        escritor_csv.writerow([titulo, fecha_publicacion, url_articulo])

# Mostrar el número total de artículos procesados
total_articulos = len(articulos)
print(f"NUMERO TOTAL DE ARTICULOS: {total_articulos}")


# Capturar el tiempo de fin de la ejecución
end_time = time.time()

# Calcular la duración
duration = end_time - start_time
print(f"Duración total de la ejecución: {duration:.2f} segundos")


Cargadas URLs y fechas desde el archivo: articulos_2024-10-22.csv
Cargadas URLs y fechas desde el archivo: articulos_2024-10-23.csv
Cargadas URLs y fechas desde el archivo: articulos_2024-10-26.csv
Cargadas URLs y fechas desde el archivo: articulos_2024-10-27.csv
Cargadas URLs y fechas desde el archivo: articulos_2024-10-28.csv
Cargadas URLs y fechas desde el archivo: articulos_2024-10-29.csv
Cargadas URLs y fechas desde el archivo: articulos_2024-11-01.csv
Cargadas URLs y fechas desde el archivo: articulos_2024-11-25.csv
Cargadas URLs y fechas desde el archivo: articulos_2024-11-26.csv
Cargadas URLs y fechas desde el archivo: articulos_2024-12-18.csv
Cargadas URLs y fechas desde el archivo: articulos_encontrados_2024-11-01.csv
Cargadas URLs y fechas desde el archivo: articulos_encontrados_2024-11-08.csv
Cargadas URLs y fechas desde el archivo: articulos_encontrados_2024-11-11.csv
Cargadas URLs y fechas desde el archivo: articulos_encontrados_2024-11-14.csv
Cargadas URLs y fechas desde

#### CON FILTRADO DE PALABRAS

In [ ]:
import csv
from datetime import datetime
import os
from urllib.parse import urlsplit, urlunsplit
from newspaper import build
import pandas as pd
import time

# Definir la carpeta donde están los archivos CSV y el archivo de Excel con palabras clave
carpeta_csv = 'archivos'
archivo_excel = 'clipping-palabras-clave.xlsx'  # Cambia esto por la ruta de tu archivo

# Obtener la fecha de hoy
hoy = datetime.now().strftime('%Y-%m-%d')

# Inicializar el contador total de artículos
total_articulos = 0

# Crear un conjunto para almacenar las URLs de los archivos CSV anteriores
urls_pasadas = set()
fecha_mas_reciente = None  # Variable para almacenar la fecha más reciente de los artículos previos
url_mas_reciente = None  # URL del artículo con la fecha más reciente
archivo_mas_reciente = None  # Nombre del archivo CSV donde se encontró la fecha más reciente

# Función para normalizar URLs eliminando el fragmento y los parámetros innecesarios
def normalizar_url(url):
    partes_url = urlsplit(url)
    url_sin_fragmento = partes_url._replace(fragment="")  # Eliminar fragmento
    return urlunsplit(url_sin_fragmento)

# Función para cargar las URLs y fechas de los archivos CSV anteriores
def cargar_urls_y_fecha_mas_reciente():
    global fecha_mas_reciente, url_mas_reciente, archivo_mas_reciente
    archivos_csv = [f for f in os.listdir(carpeta_csv) if f.endswith('.csv') and f != f'articulos_{hoy}.csv']
    for archivo_csv in archivos_csv:
        ruta_completa = os.path.join(carpeta_csv, archivo_csv)
        with open(ruta_completa, mode='r', newline='', encoding='utf-8') as archivo_csv_pasado:
            lector_csv = csv.reader(archivo_csv_pasado)
            next(lector_csv)  # Saltar la cabecera del archivo CSV
            for fila in lector_csv:
                url_normalizada = normalizar_url(fila[2])  # La URL está en la tercera columna (índice 2)
                urls_pasadas.add(url_normalizada)

                # Convertir la fecha (segunda columna) a datetime y verificar si es la más reciente
                try:
                    fecha = datetime.strptime(fila[1], '%Y-%m-%d')
                    if fecha_mas_reciente is None or fecha > fecha_mas_reciente:
                        fecha_mas_reciente = fecha
                        url_mas_reciente = url_normalizada
                        archivo_mas_reciente = archivo_csv
                except ValueError:
                    pass  # Ignorar fechas inválidas o desconocidas
        print(f"Cargadas URLs y fechas desde el archivo: {archivo_csv}")

# Cargar las URLs y la fecha más reciente de los archivos CSV anteriores
cargar_urls_y_fecha_mas_reciente()
print(f"Total de URLs cargadas desde archivos anteriores: {len(urls_pasadas)}")
print(f"Fecha más reciente en los archivos anteriores: {fecha_mas_reciente}")
print(f"URL de la fecha más reciente: {url_mas_reciente}")
print(f"Archivo CSV de la fecha más reciente: {archivo_mas_reciente}")

# Cargar palabras clave desde el archivo Excel
df = pd.read_excel(archivo_excel)
palabras = df.stack().dropna().tolist()  # Convertir a lista y eliminar NaN

# Lista temporal para almacenar los artículos antes de ordenarlos
articulos = []

# Capturar el tiempo de inicio de la ejecución
start_time = time.time()

# Recorrer todas las URLs de la lista
for url in urls:
    print(f"\nProcesando sitio: {url}")
    try:
        paper = build(url, language='es', memoize_articles=False)
        for article in paper.articles:
            url_articulo = normalizar_url(article.url)  # Normalizar la URL del artículo
            
            # Comprobar si la URL ya está en los archivos CSV anteriores
            if url_articulo in urls_pasadas:
                print(f"Artículo ya procesado previamente, ignorado: {url_articulo}")
                continue  # Si ya está en urls_pasadas, saltar este artículo

            try:
                # Descargar y procesar el artículo solo si no ha sido procesado antes
                article.download()
                article.parse()

                titulo = article.title
                fecha_publicacion = article.publish_date

                # Si la fecha de publicación es None o inválida, asignar 'Desconocido'
                if not fecha_publicacion:
                    fecha_publicacion = 'Desconocido'
                else:
                    # Convertir a string para asegurarnos de que se pueda comparar
                    fecha_publicacion = fecha_publicacion.strftime('%Y-%m-%d')

                    # Si hay una fecha más reciente registrada y este artículo es más antiguo, ignorarlo
                    # Pero si el artículo tiene la misma fecha que la más reciente, procesarlo.
                    if fecha_mas_reciente and datetime.strptime(fecha_publicacion, '%Y-%m-%d') < fecha_mas_reciente:
                        print(f"Artículo ignorado porque es más antiguo que la fecha más reciente registrada.")
                        continue

                # Comprobar si el título contiene alguna de las palabras clave y almacenar todas las coincidencias
                palabras_encontradas = [palabra for palabra in palabras if palabra.lower() in titulo.lower()]
                if palabras_encontradas:
                    articulos.append((titulo, fecha_publicacion, url_articulo, palabras_encontradas))  # URL en formato de lista

            except Exception as e:
                print(f"Error procesando el artículo: {e}")

    except Exception as e:
        print(f"Error procesando el sitio: {e}")

# Ordenar los artículos, manejando correctamente 'Desconocido' para que se vaya al final
articulos.sort(key=lambda x: datetime.strptime(x[1], '%Y-%m-%d') if x[1] != 'Desconocido' else datetime.max)

# Crear el archivo CSV de hoy y abrirlo en modo escritura
archivo_hoy = os.path.join(carpeta_csv, f'articulos_encontrados_{hoy}.csv')
with open(archivo_hoy, mode='w', newline='', encoding='utf-8') as archivo_csv_hoy:
    escritor_csv = csv.writer(archivo_csv_hoy)
    escritor_csv.writerow(['Título', 'Fecha de Publicación', 'URL', 'Palabras Encontradas'])

    for titulo, fecha_publicacion, url_articulo, palabras_encontradas in articulos:
        escritor_csv.writerow([titulo, fecha_publicacion, url_articulo, palabras_encontradas])

# Mostrar el número total de artículos procesados
total_articulos = len(articulos)
print(f"NUMERO TOTAL DE ARTICULOS: {total_articulos}")

# Capturar el tiempo de fin de la ejecución
end_time = time.time()

# Calcular la duración
duration = end_time - start_time
print(f"Duración total de la ejecución: {duration:.2f} segundos")


Cargadas URLs y fechas desde el archivo: articulos_2024-10-22.csv
Cargadas URLs y fechas desde el archivo: articulos_2024-10-23.csv
Cargadas URLs y fechas desde el archivo: articulos_2024-10-26.csv
Cargadas URLs y fechas desde el archivo: articulos_2024-10-27.csv
Cargadas URLs y fechas desde el archivo: articulos_2024-10-28.csv
Cargadas URLs y fechas desde el archivo: articulos_2024-10-29.csv
Cargadas URLs y fechas desde el archivo: articulos_2024-11-01.csv
Cargadas URLs y fechas desde el archivo: articulos_2024-11-25.csv
Cargadas URLs y fechas desde el archivo: articulos_encontrados_2024-11-01.csv
Cargadas URLs y fechas desde el archivo: articulos_encontrados_2024-11-08.csv
Cargadas URLs y fechas desde el archivo: articulos_encontrados_2024-11-11.csv
Cargadas URLs y fechas desde el archivo: articulos_encontrados_2024-11-14.csv
Cargadas URLs y fechas desde el archivo: articulos_encontrados_2024-11-25.csv
Cargadas URLs y fechas desde el archivo: articulos_lavoz_2024-11-08.csv
Total de U

In [32]:
archivo_excel = 'clipping-palabras-clave.xlsx'  # Cambia esto por la ruta de tu archivo

df=pd.read_excel(archivo_excel)
df = df.iloc[:, 1:]  # Mantener todas las filas/columnas excepto las primeras
# Crear el primer grupo: palabras de las primeras dos columnas
grupo_especificas = df.iloc[:, :2].stack().dropna().tolist()

# Crear el segundo grupo: palabras de las columnas restantes
grupo_genericas = df.iloc[:, 2:].stack().dropna().tolist()

# Mostrar los grupos
print("Grupo específico (primeras dos columnas):", grupo_especificas)
print("Grupo genérico (resto de las columnas):", grupo_genericas)


# Función para extraer palabras separadas por "o" usando re.split()
def extraer_palabras(texto):
    if pd.isna(texto):  # Si el valor es NaN, no hacer nada
        return []
    # Usar re.split() para dividir por " o " y devolver una lista de palabras
    return re.split(r'\s+o\s+', str(texto))  # Dividir por la palabra "o" rodeada de espacios

# Ejemplo de uso
print(extraer_palabras("hola"))
palabras_extraidas=[]
# Recorrer todas las celdas del DataFrame
for i in range(df.shape[0]):  # Recorrer filas
    for j in range(df.shape[1]):  # Recorrer columnas
        # Extraer las palabras de cada celda
        palabras = extraer_palabras(df.iloc[i, j])
        if palabras:  # Si se encuentran palabras, añadirlas a la lista
            palabras_extraidas.append(palabras[0])  # Guardamos las dos palabras extraídas

# Mostrar las palabras extraídas
print(palabras_extraidas)

Grupo específico (primeras dos columnas): ['Universidad de Oviedo', 'EPI Gijón ', 'Universidá d’Uviéu', 'Escuela Politécnica de Ingeniería de Gijón', 'Rector de la Universidad de Oviedo', 'Escuela Politécnica de Ingeniería ', 'Ignacio Villaverde', 'Escuela Politécnica de Mieres', 'UniOvi', 'Facultad Jovellanos', 'Vicerrectorado de Investigación', 'Escuela de Minas', 'Irene Díaz', 'Escuela de la Marina Civil', 'Vicerrectorado de Profesorado ', 'Facultad de Medicina', 'Pedro Alonso', 'Facultad de Biología', 'Vicerrectorado de Extensión', 'Facultad de Ciencias', 'Pilar García ', 'Facultad de Comercio, Turismo y Ciencias Sociales Jovellanos', 'Vicerrectorado de Estudiantes', 'Facultad de Derecho', 'Alfonso López ', 'Facultad de Economía y Empresa', 'Vicerrectorado de Transferencia', 'Facultad de Filosofía y Letras', 'Susana Luque', 'Facultad de Formación del Profesorado y Educación', 'Vicerrectorado de Sostenibilidad', 'Facultad de Geología', 'Inés Peñuelas', 'Facultad de Medicina y Cienci